In [127]:
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_airports_in_China'

table = pd.read_html(url, header=0,keep_default_na=False) 

tdf = table[0]
tdf = tdf.drop('ICAO', 1)

tdf.columns = ['City','Code','Airport']

tdf = tdf.query('Airport != ""').reset_index(drop=True)
tdf = tdf.query('Code != ""').reset_index(drop=True)

airports=tdf
airports.set_index('Code', inplace=True)

#order = ['Airport', 'City', 'Code']
#airports = airports[order]
airports.head(10)

#airports.shape



,City,Airport
Code,,
AQG,Anqing,Anqing Tianzhushan Airport
JUH,Chizhou,Chizhou Jiuhuashan Airport
FUG,Fuyang,Fuyang Xiguan Airport
HFE,Hefei,Hefei Xinqiao International Airport
TXN,Huangshan,Huangshan Tunxi International Airport
PEK,Beijing,Beijing Capital International Airport
NAY,Beijing,Beijing Nanyuan Airport
CKG,Chongqing,Chongqing Jiangbei International Airport
JIQ,Qianjiang,Qianjiang Wulingshan Airport


In [128]:
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge


In [129]:
from geopy.geocoders import Nominatim

In [130]:
'''
# define the dataframe columns
column_names = ['Code','Longitude','Latitude'] 

# instantiate the dataframe
airports_coordinates = pd.DataFrame(columns=column_names)

geolocator = Nominatim(user_agent="my-application1")

for row_index, row in airports.iterrows():
    address = row[1]
    print(row_index, row[0],row[1])
    location = geolocator.geocode(address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude =0
        longitude=0
    airports_coordinates = airports_coordinates.append({'Code': row_index,
                    'Latitude': latitude,'Longitude':longitude}, ignore_index=True) 

airports_coordinates.set_index('Code', inplace=True)

airports_coordinates.to_csv("airports_coordinates.csv",index=True,sep=',')
print("Done")
'''

'\n# define the dataframe columns\ncolumn_names = [\'Code\',\'Longitude\',\'Latitude\'] \n\n# instantiate the dataframe\nairports_coordinates = pd.DataFrame(columns=column_names)\n\ngeolocator = Nominatim(user_agent="my-application1")\n\nfor row_index, row in airports.iterrows():\n    address = row[1]\n    print(row_index, row[0],row[1])\n    location = geolocator.geocode(address)\n    if location:\n        latitude = location.latitude\n        longitude = location.longitude\n    else:\n        latitude =0\n        longitude=0\n    airports_coordinates = airports_coordinates.append({\'Code\': row_index,\n                    \'Latitude\': latitude,\'Longitude\':longitude}, ignore_index=True) \n\nairports_coordinates.set_index(\'Code\', inplace=True)\n\nairports_coordinates.to_csv("airports_coordinates.csv",index=True,sep=\',\')\nprint("Done")\n'

In [131]:
airports.head()

,City,Airport
Code,,
AQG,Anqing,Anqing Tianzhushan Airport
JUH,Chizhou,Chizhou Jiuhuashan Airport
FUG,Fuyang,Fuyang Xiguan Airport
HFE,Hefei,Hefei Xinqiao International Airport
TXN,Huangshan,Huangshan Tunxi International Airport


In [132]:
airports_coordinates = pd.read_csv('airports_coordinates.csv')
airports_coordinates.set_index('Code', inplace=True)
airports_coordinates.head()


,Longitude,Latitude
Code,,
AQG,117.046532,30.581347
JUH,117.685837,30.740428
FUG,115.734768,32.882703
HFE,116.967582,31.988717
TXN,118.255246,29.733764


In [133]:
airports_coordinates.shape

(237, 2)

In [134]:
airports.shape

(237, 2)

In [135]:

airport_result = pd.merge(pd.DataFrame(airports),pd.DataFrame(airports_coordinates[['Latitude','Longitude']]),left_index=True,right_index=True)
airport_result.head()

#airport_result = pd.merge(airports,
#                     airports_coordinates[['Latitude','Longitude']],
#                     on='Code')
#airport_result.head()

,City,Airport,Latitude,Longitude
Code,,,,
AQG,Anqing,Anqing Tianzhushan Airport,30.581347,117.046532
JUH,Chizhou,Chizhou Jiuhuashan Airport,30.740428,117.685837
FUG,Fuyang,Fuyang Xiguan Airport,32.882703,115.734768
HFE,Hefei,Hefei Xinqiao International Airport,31.988717,116.967582
TXN,Huangshan,Huangshan Tunxi International Airport,29.733764,118.255246


In [201]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [137]:
airport_result.reset_index(inplace=True)
airport_result.head()

,Code,City,Airport,Latitude,Longitude
0,AQG,Anqing,Anqing Tianzhushan Airport,30.581347,117.046532
1,JUH,Chizhou,Chizhou Jiuhuashan Airport,30.740428,117.685837
2,FUG,Fuyang,Fuyang Xiguan Airport,32.882703,115.734768
3,HFE,Hefei,Hefei Xinqiao International Airport,31.988717,116.967582
4,TXN,Huangshan,Huangshan Tunxi International Airport,29.733764,118.255246


In [138]:
def getNearbyVenues(names, latitudes, longitudes, city, code, categoryid, radius=1000):
    
    venues_list=[]
    for name, lat, lng, cty, psg in zip(names, latitudes, longitudes, city, code):
        print("Airport name: " + name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryid)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            cty,
            psg,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Airport', 
                  'Airport Latitude', 
                  'Airport Longitude', 
                  'City',
                  'Code',           
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [143]:
airport_result

,Code,City,Airport,Latitude,Longitude
0,AQG,Anqing,Anqing Tianzhushan Airport,30.581347,117.046532
1,JUH,Chizhou,Chizhou Jiuhuashan Airport,30.740428,117.685837
2,FUG,Fuyang,Fuyang Xiguan Airport,32.882703,115.734768
3,HFE,Hefei,Hefei Xinqiao International Airport,31.988717,116.967582
4,TXN,Huangshan,Huangshan Tunxi International Airport,29.733764,118.255246
5,PEK,Beijing,Beijing Capital International Airport,40.079285,116.594561
6,NAY,Beijing,Beijing Nanyuan Airport,39.777220,116.387983
7,CKG,Chongqing,Chongqing Jiangbei International Airport,29.726159,106.649836
8,JIQ,Qianjiang,Qianjiang Wulingshan Airport,29.515216,108.830895
9,WXN,Wanzhou,Wanzhou Wuqiao Airport,30.801289,108.434805


In [146]:

airport_result = airport_result.query('Latitude != "0.000000"').reset_index(drop=True)
airport_result = airport_result.query('Longitude != "0.000000"').reset_index(drop=True)




In [147]:
airport_result

,Code,City,Airport,Latitude,Longitude
0,AQG,Anqing,Anqing Tianzhushan Airport,30.581347,117.046532
1,JUH,Chizhou,Chizhou Jiuhuashan Airport,30.740428,117.685837
2,FUG,Fuyang,Fuyang Xiguan Airport,32.882703,115.734768
3,HFE,Hefei,Hefei Xinqiao International Airport,31.988717,116.967582
4,TXN,Huangshan,Huangshan Tunxi International Airport,29.733764,118.255246
5,PEK,Beijing,Beijing Capital International Airport,40.079285,116.594561
6,NAY,Beijing,Beijing Nanyuan Airport,39.777220,116.387983
7,CKG,Chongqing,Chongqing Jiangbei International Airport,29.726159,106.649836
8,JIQ,Qianjiang,Qianjiang Wulingshan Airport,29.515216,108.830895
9,WXN,Wanzhou,Wanzhou Wuqiao Airport,30.801289,108.434805


In [145]:
import requests

food_category_id = '4d4b7105d754a06374d81259'

airport_venues = getNearbyVenues(names=airport_result['Airport'],
                                   latitudes=airport_result['Latitude'],longitudes=airport_result['Longitude'],
                                   city=airport_result['City'],code=airport_result['Code'],
                                   categoryid=food_category_id)

Airport name: Anqing Tianzhushan Airport
Airport name: Chizhou Jiuhuashan Airport
Airport name: Fuyang Xiguan Airport
Airport name: Hefei Xinqiao International Airport
Airport name: Huangshan Tunxi International Airport
Airport name: Beijing Capital International Airport
Airport name: Beijing Nanyuan Airport
Airport name: Chongqing Jiangbei International Airport
Airport name: Qianjiang Wulingshan Airport
Airport name: Wanzhou Wuqiao Airport
Airport name: Fuzhou Changle International Airport
Airport name: Quanzhou Jinjiang International Airport
Airport name: Sanming Shaxian Airport
Airport name: Wuyishan Airport
Airport name: Xiamen Gaoqi International Airport
Airport name: Dunhuang Airport
Airport name: Jiayuguan Airport
Airport name: Jinchang Jinchuan Airport
Airport name: Lanzhou Zhongchuan International Airport
Airport name: Longnan Chengzhou Airport
Airport name: Qingyang Airport
Airport name: Tianshui Maijishan Airport
Airport name: Gannan Xiahe Airport
Airport name: Zhangye Ganzh

In [150]:
print(airport_venues.shape)
airport_venues.head()


(259, 9)


,Airport,Airport Latitude,Airport Longitude,City,Code,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hefei Xinqiao International Airport,31.988717,116.967582,Hefei,HFE,徽味縁,31.988706,116.965124,Comfort Food Restaurant
1,Hefei Xinqiao International Airport,31.988717,116.967582,Hefei,HFE,KFC (肯德基),31.988972,116.964681,Fast Food Restaurant
2,Huangshan Tunxi International Airport,29.733764,118.255246,Huangshan,TXN,龙华食府,29.727752,118.260665,Chinese Restaurant
3,Beijing Capital International Airport,40.079285,116.594561,Beijing,PEK,KFC (肯德基),40.075864,116.589118,Fast Food Restaurant
4,Beijing Capital International Airport,40.079285,116.594561,Beijing,PEK,Tai Hing Restaurant 太興茶餐廳,40.080972,116.588093,Cantonese Restaurant


In [151]:
airport_venues.groupby('Airport').count()

,Airport Latitude,Airport Longitude,City,Code,Venue,Venue Latitude,Venue Longitude,Venue Category
Airport,,,,,,,,
Baise Bama Airport,1,1,1,1,1,1,1,1
Beijing Capital International Airport,18,18,18,18,18,18,18,18
Changchun Longjia International Airport,4,4,4,4,4,4,4,4
Changsha Huanghua International Airport,4,4,4,4,4,4,4,4
Changzhou Benniu Airport,1,1,1,1,1,1,1,1
Dali Airport,2,2,2,2,2,2,2,2
Dalian Zhoushuizi International Airport,5,5,5,5,5,5,5,5
Dehong Mangshi Airport,1,1,1,1,1,1,1,1
Dêqên Shangri-La Airport,2,2,2,2,2,2,2,2


In [152]:
print('There are {} uniques categories.'.format(len(airport_venues['Venue Category'].unique())))

There are 42 uniques categories.


In [153]:
# one hot encoding
airport_onehot = pd.get_dummies(airport_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
airport_onehot['Airport'] = airport_venues['Airport'] 

# move Airport column to the first column
fixed_columns = [airport_onehot.columns[-1]] + list(airport_onehot.columns[:-1])
airport_onehot = airport_onehot[fixed_columns]

airport_onehot.head()

,Airport,Asian Restaurant,Bakery,Bistro,Buffet,Burger Joint,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,...,Shaanxi Restaurant,Shandong Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Tibetan Restaurant,Xinjiang Restaurant,Yunnan Restaurant
0,Hefei Xinqiao International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hefei Xinqiao International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Huangshan Tunxi International Airport,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Beijing Capital International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beijing Capital International Airport,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
airport_onehot.shape


(259, 43)

In [155]:
airport_grouped = airport_onehot.groupby('Airport').mean().reset_index()
airport_grouped

,Airport,Asian Restaurant,Bakery,Bistro,Buffet,Burger Joint,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,...,Shaanxi Restaurant,Shandong Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Tibetan Restaurant,Xinjiang Restaurant,Yunnan Restaurant
0,Baise Bama Airport,0.000000,1.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
1,Beijing Capital International Airport,0.055556,0.055556,0.055556,0.000000,0.000000,0.00,0.166667,0.055556,0.055556,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
2,Changchun Longjia International Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
3,Changsha Huanghua International Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.750000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
4,Changzhou Benniu Airport,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
5,Dali Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.5
6,Dalian Zhoushuizi International Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.400000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
7,Dehong Mangshi Airport,1.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0
8,Dêqên Shangri-La Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.5,0.00,0.0
9,Fuzhou Changle International Airport,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.222222,...,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0


In [156]:
airport_grouped.shape

(56, 43)

In [157]:
num_top_venues = 5

for hood in airport_grouped['Airport']:
    print("----"+hood+"----")
    temp = airport_grouped[airport_grouped['Airport'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baise Bama Airport----
                  venue  freq
0                Bakery   1.0
1      Asian Restaurant   0.0
2    Seafood Restaurant   0.0
3   Macanese Restaurant   0.0
4  Mongolian Restaurant   0.0


----Beijing Capital International Airport----
                  venue  freq
0                  Café  0.17
1  Fast Food Restaurant  0.17
2         Deli / Bodega  0.11
3   Japanese Restaurant  0.11
4      Asian Restaurant  0.06


----Changchun Longjia International Airport----
                  venue  freq
0          Noodle House   0.5
1                  Café   0.5
2      Asian Restaurant   0.0
3    Shaanxi Restaurant   0.0
4  Mongolian Restaurant   0.0


----Changsha Huanghua International Airport----
                  venue  freq
0    Chinese Restaurant  0.75
1  Fast Food Restaurant  0.25
2      Asian Restaurant  0.00
3    Seafood Restaurant  0.00
4  Mongolian Restaurant  0.00


----Changzhou Benniu Airport----
                     venue  freq
0         Asian Restaurant   1.0
1   

                  venue  freq
0  Fast Food Restaurant  0.50
1      Asian Restaurant  0.17
2          Noodle House  0.17
3        Sandwich Place  0.17
4    Seafood Restaurant  0.00


----Shanghai Hongqiao International Airport----
                  venue  freq
0  Fast Food Restaurant  0.21
1          Noodle House  0.16
2                  Café  0.16
3   Japanese Restaurant  0.11
4    Chinese Restaurant  0.11


----Shanghai Pudong International Airport----
                venue  freq
0    Asian Restaurant  0.25
1                Café  0.25
2         Snack Place  0.25
3   Korean Restaurant  0.25
4  Seafood Restaurant  0.00


----Shenyang Taoxian International Airport----
                     venue  freq
0     Fast Food Restaurant  0.50
1       Dim Sum Restaurant  0.17
2                     Café  0.17
3       Chinese Restaurant  0.17
4  New American Restaurant  0.00


----Shenzhen Bao'an International Airport----
                  venue  freq
0    Italian Restaurant   0.2
1                  

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [161]:
import numpy as np
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Airport']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
airport_venues_sorted = pd.DataFrame(columns=columns)
airport_venues_sorted['Airport'] = airport_grouped['Airport']

for ind in np.arange(airport_grouped.shape[0]):
    airport_venues_sorted.iloc[ind, 1:] = return_most_common_venues(airport_grouped.iloc[ind, :], num_top_venues)

airport_venues_sorted

,Airport,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Baise Bama Airport,Bakery,Yunnan Restaurant,Deli / Bodega
1,Beijing Capital International Airport,Café,Fast Food Restaurant,Japanese Restaurant
2,Changchun Longjia International Airport,Noodle House,Café,Yunnan Restaurant
3,Changsha Huanghua International Airport,Chinese Restaurant,Fast Food Restaurant,Yunnan Restaurant
4,Changzhou Benniu Airport,Asian Restaurant,Deli / Bodega,Irish Pub
5,Dali Airport,Yunnan Restaurant,Café,Deli / Bodega
6,Dalian Zhoushuizi International Airport,Fast Food Restaurant,Café,Sandwich Place
7,Dehong Mangshi Airport,Asian Restaurant,Deli / Bodega,Irish Pub
8,Dêqên Shangri-La Airport,Restaurant,Tibetan Restaurant,Yunnan Restaurant
9,Fuzhou Changle International Airport,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant


In [162]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  50.27 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.18 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.33 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  36.97 MB/s
Libraries imported.


In [165]:
airport_grouped.head()

,Airport,Asian Restaurant,Bakery,Bistro,Buffet,Burger Joint,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,...,Shaanxi Restaurant,Shandong Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Swiss Restaurant,Thai Restaurant,Tibetan Restaurant,Xinjiang Restaurant,Yunnan Restaurant
0,Baise Bama Airport,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Beijing Capital International Airport,0.055556,0.055556,0.055556,0.0,0.0,0.0,0.166667,0.055556,0.055556,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Changchun Longjia International Airport,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.500000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Changsha Huanghua International Airport,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Changzhou Benniu Airport,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
# set number of clusters
kclusters = 3

airport_grouped_clustering = airport_grouped.drop('Airport', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(airport_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int32)

In [183]:
airport_venues_sorted.head()

,Airport,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Baise Bama Airport,Bakery,Yunnan Restaurant,Deli / Bodega
1,Beijing Capital International Airport,Café,Fast Food Restaurant,Japanese Restaurant
2,Changchun Longjia International Airport,Noodle House,Café,Yunnan Restaurant
3,Changsha Huanghua International Airport,Chinese Restaurant,Fast Food Restaurant,Yunnan Restaurant
4,Changzhou Benniu Airport,Asian Restaurant,Deli / Bodega,Irish Pub


In [188]:
airport_merged

,Code,City,Airport,Latitude,Longitude
0,AQG,Anqing,Anqing Tianzhushan Airport,30.581347,117.046532
1,JUH,Chizhou,Chizhou Jiuhuashan Airport,30.740428,117.685837
2,FUG,Fuyang,Fuyang Xiguan Airport,32.882703,115.734768
3,HFE,Hefei,Hefei Xinqiao International Airport,31.988717,116.967582
4,TXN,Huangshan,Huangshan Tunxi International Airport,29.733764,118.255246
5,PEK,Beijing,Beijing Capital International Airport,40.079285,116.594561
6,NAY,Beijing,Beijing Nanyuan Airport,39.777220,116.387983
7,CKG,Chongqing,Chongqing Jiangbei International Airport,29.726159,106.649836
8,JIQ,Qianjiang,Qianjiang Wulingshan Airport,29.515216,108.830895
9,WXN,Wanzhou,Wanzhou Wuqiao Airport,30.801289,108.434805


In [200]:
airport_merged = airport_result

# add clustering labels
#airport_merged['Cluster Labels'] = kmeans.labels_
airport_venues_sorted['Cluster Labels'] = kmeans.labels_
#print(airport_merged.head())
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#airport_merged = airport_merged.join(airport_venues_sorted.set_index('Airport'), on='Airport')

result = airport_venues_sorted.join(airport_merged.set_index('Airport'), on='Airport')
airport_merged=result


#airport_merged.set_index('Airport')
#airport_venues_sorted.set_index('Airport')
#airport_merged = pd.join(pd.DataFrame(airport_merged),pd.DataFrame(airport_venues_sorted),how='right',left_index=True,right_index=True)




airport_merged # check the last columns!

,Airport,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Cluster Labels,Code,City,Latitude,Longitude
0,Baise Bama Airport,Bakery,Yunnan Restaurant,Deli / Bodega,0,AEB,Baise,45.222541,-81.631701
1,Beijing Capital International Airport,Café,Fast Food Restaurant,Japanese Restaurant,0,PEK,Beijing,40.079285,116.594561
2,Changchun Longjia International Airport,Noodle House,Café,Yunnan Restaurant,0,CGQ,Changchun,43.996073,125.686564
3,Changsha Huanghua International Airport,Chinese Restaurant,Fast Food Restaurant,Yunnan Restaurant,1,CSX,Changsha,28.190133,113.220895
4,Changzhou Benniu Airport,Asian Restaurant,Deli / Bodega,Irish Pub,2,CZX,Changzhou,31.913288,119.782588
5,Dali Airport,Yunnan Restaurant,Café,Deli / Bodega,0,DLU,Dali,25.650167,100.319518
6,Dalian Zhoushuizi International Airport,Fast Food Restaurant,Café,Sandwich Place,0,DLC,Dalian,38.964101,121.541565
7,Dehong Mangshi Airport,Asian Restaurant,Deli / Bodega,Irish Pub,2,LUM,"Mangshi, Dehong",24.397945,98.528252
8,Dêqên Shangri-La Airport,Restaurant,Tibetan Restaurant,Yunnan Restaurant,0,DIG,"Shangri-La, Dêqên",27.792296,99.678951
9,Fuzhou Changle International Airport,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,0,FOC,Fuzhou,25.935096,119.666656


In [197]:
airport_merged[['Latitude']] = airport_merged[['Latitude']].astype(float)
airport_merged[['Longitude']] = airport_merged[['Longitude']].astype(float)


In [198]:
latitude = 120.0
longitude = 35.0
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=2)


In [199]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(airport_merged['Latitude'], airport_merged['Longitude'], airport_merged['Airport'], airport_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters